![](nlpday_logo.png)

# Rozpoznawanie mowy dla NLP

**Danijel Koržinek** <danijel@pja.edu.pl>

## Abstrakt

Technologia automatycznego rozpoznawania mowy (ASR) się staje coraz bardziej powszechna w naszym życiu, a w związku tym wzrasta i potrzeba jej integracji z innymi rozwiązaniami w środowisku informatycznym. Nieodłącznym komponentem tej układanki jest analiza wypowiedzi szeregiem algorytmów, powszechnie stosowanych w przetwarzaniu języka naturalnego, ale coraz częściej się dowiadujemy, że w praktyce nie jest to takie proste i skuteczne, gdyż większość narzędzi i modeli nie jest dostosowana do specyfiki języka mówionego. Problem ten nie jest nowością i pojawia się dosyć często w innych zadaniach, szczególnie gdy w grę wchodzi analiza komunikacji międzyludzkiej, np. w czatach, czy forach internetowych.

Zadaniem tych warsztatów nie jest dogłębna analiza problemu rozpoznawania mowy, ale zaprezentowanie pewnego rozwiązania typu opensource umożliwiającego szybkie i tanie stosowanie technologii ASR w sposób w pełni konfigurowalny w celu wygenerowania wiarygodnego wyniku procesu rozpoznawania mowy nadającego się do dalszych badań w kontekście NLP. Zaletą tego podejścia od stosowania gotowych rozwiązań chmurowych jest pełna kontrola nad każdym aspektem procesu i możliwość jego modyfikacji. Zastosowano w nim mechanizm hybrydowy, który umożliwi łatwą modyfikację słownictwa i warstwy językowej bez kosztownego dotrenowywania modeli end-to-end.

Warsztat ten wyjaśni w skrócie działanie procesu ASR pomijając przy tym szczegóły, które się nie mieszczą w ramach czasowych warsztatu. Zostanie użyty gotowy, pre-trenowany model akustyczny i dostarczone gotowe próbki nagrań. Warsztat się kończy wygenerowaniem wyniku rozpoznawania mowy na różne sposoby, a zagadnienia dotyczące analizy tego wyniku pod kątem NLP są zostawione uczestnikom, jako temat do osobistych rozważań i eksploracji.

![](masterclass.png)

## Wprowadzenie

W tych warsztatach poznasz proces ASR przetwarzającego nagranie audio mowy do transkrypcji ortograficzenj w postaci sekwencji słów. W tym celu użyjemy dane pochodzące z posiedzeń Polskiego Sejmu. Główny nacisk omawianych tematów zostanie położony na aspektach językowych całego procesu, a przetwarzanie akustyki zostanie sprowadzone do minimum. W tym celu użyjemy gotowego, wytrenowanego modelu akustycznego, ale poświęcimy czas na stworzenie modelu jęyzka od postaw.

Do realizacji zadania wykorzystamy hybrydowy system rozpoznawania oparty o WFST z projektu typu opensource [Kaldi](https://kaldi-asr.org). Najpopularniejsze systemy ASR rozróżńiamy ogólnie na następujące rodzaje:

1. tradycyjne - oparte o ukryte modele Markova (HMM) z modelem akustycznycm wykorzystującym mieszkanki Gaussowskie (GMM)
2. hybrydowe - model akustyczny oparty o głęboką sieć neuronową połączoną z HMM lub WFST do modelowania sekwencji wyrazów
3. end-to-end (E2E) - system gdzie wszystkie komponenty są modelowane siecią neuronową

Rozwiązania typu E2E są kosztowne ze względu na wymagane zasoby, czas i moc obliczeniową, ale oprócz tego generują model, który jest niełatwy w modyfikacji i dostosowywaniu do konkretnych potrzeb. Systemy hybrydowe są dobrym kompromisem między możliwością dostosowywania, a jakością wyniku. Przykładowo, łatwo można dodać nowe wyrazy bez konieczności wytrenowania całego systemu od podstaw.

## Przygotowanie środowiska

Zaczniemy od ściągnięcia zestawu programów z pakietu systemu Kaldi, skompilowanych żeby się uruchamiały w środowisku Colab. Więcej informacji o zawartości jest [tutaj](https://github.com/danijel3/ASRforNLP/releases/tag/v1.0). Poniższy blok ściąga programy i bibiloteki i umieszcza ich w odpowiednim miejscu żeby były widoczne w systemie.

In [ ]:
!wget https://github.com/danijel3/ASRforNLP/releases/download/v1.0/kaldi.tar.xz

!tar xvf kaldi.tar.xz -C / > /dev/null
%rm kaldi.tar.xz

!for f in $(find /opt/kaldi -name *.so*) ; do ln -sf $f /usr/local/lib/$(basename $f) ; done
!for f in $(find /opt/kaldi/src -not -name *.so* -type f -executable) ; do ln -s $f /usr/local/bin/$(basename $f) ; done
!for f in $(find /opt/kaldi/tools -not -name *.so* -type f -executable) ; do ln -s $f /usr/local/bin/$(basename $f) ; done

!ldconfig

Następnie ściągniemy przykładowe nagrania i zbiór tekstów do trenowania modelu języka.

In [ ]:
!wget https://github.com/danijel3/ASRforNLP/releases/download/v1.1/sejm-audio.tar.xz
!wget https://github.com/danijel3/ASRforNLP/releases/download/v1.1/sejm-text.xz

!tar xvf sejm-audio.tar.xz > /dev/null
!xz -d sejm-text.xz

%rm sejm-audio.tar.xz

Możemy obejrzeć, że katalog zawiera zestaw plików w formacie WAV i plik `text` zawierający referencyjną transkrypcję wszystkich nagrań:

Następnie dokonamy odsłuchu audio. Należy zaimportować komponent `Audio` z modułu `IPython.display`, a potem uruchomić funkcję `Audio()` podając jej ścieżkę do pliku w katalogu `sejm-audio`:

Następnie pobierzemy wytrenowany model akustyczny i model do tranksrypcji fonetycznej. Udostępnione modele utworzono w ramach projektu [Clarin-PL](https://www.clarin-pl.eu). Dostępne są również inne modele, więc proszę o kontkat w razie dodatkowych pytań.

In [ ]:
!wget https://github.com/danijel3/ASRforNLP/releases/download/v1.2/models.tar.xz

!tar xvf models.tar.xz > /dev/null

%rm models.tar.xz

W podejściu hybrydowym proces rozpoznawania mowy podzielono na etapy:

1. wczytanie sygnału audio i ekstrakcja cech akustycznych
2. modelowanie akustyczne oceniające prawodopodobieństwo występowania różych jednostek fonetycznych lub językowych w sygnale audio
3. generowanie i ocena hipotez sekwencji słów pasujących do prawdopodobieństw z pkt. 2
4. wybór najbardziej wiarygodnej sekwencji z pkt. 3

W praktyce wszystkie te etapy są wykonywane równocześnie (synchronicznie), ale koncepcyjnie podział ten jest zawsze widoczny. Można sobie zadać pytanie, czym są dokładnie dane przechodzące między punktami 1->2 i 2->3. Jeśli chodzi o cechy akustyczne, nie będzie to omawiane w wielkich szczegółach na tych warsztatach, ale zainteresowani na pewno łatwo znajdą taki opis w itnernecie, np [ten](https://github.com/danijel3/PyHTK/blob/master/python-notebooks/HTKFeaturesExplained.ipynb). 

Jeśli chodzi o jednostki fonetyczne lub językowe, tutaj można znaleźć różne podejścia. Wiele metod E2E próbuje ten problem rozwiązać modelując litery (zapisu ortograficzengo) lub fonemy (czyli jednostki wymowy), a ostatnio nawet fragmenty słów (tzw. sub-word units, wordpiece, sentencepiece, BPE, czy przynajmniej sylaby). Idealnie byłoby rozpoznawać bezpośrednio słowa, ale trudno sobie wyobrazić sieć neuronową, która ma kilkaset tysięcy, czy nawet kilka milionów neuronów w warstwie softmax. Podejście tradycyjne (i przez to również hybrydowe) jest jednak nieco bardziej rozbudowane:

1. na samym początku każdy wyraz zamieniamy na sekwencję fonemów używając mapowania słów zwanych **leksykonem**. Jest strudktura która zamienia każdy wyraz ze słownika (w postaci *string*) na listę fonemów. Możliwe są też alternatywne wymowy (czyli ten sam wyraz ma kilka możliwych transkrypcji). Wygenerowanie takiego słownika jest możliwe w sposób automatyczny na podstawie metody zwanej G2P (grapheme-to-phoneme).

2. każdy fonem nie modelujemy bezpośrednio, ale napodstawie jego kontekstu. Jedostka ta nazywa się **trifonem**, czyli fonem który ma kontekst lewo- i prawo-stronny innego fonemu. Np. `o-l+a`, to trifon `l` który z lewej strony ma fonem `o`, a z prawej `a`.

3. dodatkowo, każdy trifon jest modelowany maszyną stanów modelującą (zazwyczaj) 3 fazy: początkową, środkową i końcową. Te "ukryte" stany, są znane pod nazwą (ang.) **senone** i mogą być dzielone z innymi stanami w różnych trifonach. Podejście to jest znane jako *state tying* i służy ograniczeniu ich ilości.

Podsumowując, model akustyczny jest używany do modelowania poszczególnych stanów ukrytych (**H**) zwanych *senonami*, które razem modelują poszczególne kontekstowe jednostki fonetyczne (**C**) zwane *trifonami*, które można na postawie leksykonu (**L**) zamienić z fonemów na pełne wyrazy. 

W pewnym sensie, można od razu przejść z modelu akustycznego bezpośrednio do poszcególych wyrazów. Zostaje nam tylko jeden element, czyli modelowanie sekwencji wyrazów. W podejściu hybrydowym, modelujemy go używając maszyny stanów zwanej fachowo gramatyką formalną (**G**). Jest kilka sposobów na zaprojektowanie takiej gramatyki: można to zrobić ręcznie definując bezpośrednio dozwolone sekwencje wyrazów, lub automatycznie na przykład używając analizy statystycznej dużego zbioru tekstów. W kolejnych rozdziałach opiszemy po kolei obydwa podejścia.

## Podejście oparte na gramatyce skończonej

Jeśli chcemy ręcznie zdefiniować jakie sekwencje wyrazów ma rozpoznawać nasz ASR, możemy do tego celu zdefiniować gramatykę skończoną. Używamy do tego języka opisu przypominającego znane wszystkim wyrażenia regularne (tzw. regexpy). W komercyjnych zastosowaniach stosuje się standard [SRGS](https://www.w3.org/TR/speech-grammar/) i język opisów ABNF lub GRXML. System Kaldi jest oparty o mechanizm WFST i wykorzystuje do tego świetną bibliotekę [OpenFST](https://www.openfst.org/twiki/bin/view/FST/WebHome) (dla osób zainteresowanych ogólnie NLP, polecam sprawdzić jej możliwości, jeśli jej nie znacie). My zatem zdefiniujemy gramatykę używając formatu FST i do tego nam się przyda biblioteka `openfst-python`. Użyj następującego polecenia żeby zainstalować bibliotekę:

```
!pip install openfst-python
```

### Przykład atuomatu

Każde WFST jest zdefiniowane następującymi elementami:

- zbiór stanów
- zbiór symboli wejściowych
    - z względu na wydajność oblieczniową każdy symbol to para (`string`,`int`), gdzie `int` jest używany w modelu, a konwertowany do `string` tylko do wizualizacji/wydruku
- zbiór symboli wyjściowych
    - j/w
- połączenia między stanami
    - połączenia to jednokierunkowe relacje między dwoma stanami $s_a \rightarrow s_b$
    - każde połączenie jest zdefiniowane trójką (symbol wejśćiowy, symbol wyjściowy, waga)
    - waga może być zdefniowana na wiele sposób, albo zostać pominięta

Najpierw trzeba zaimportować bibliotkę `openfst_python` i najlepiej od razu skrócić jej nazwę poleceniem `import openfst_python as fst`. Potem zacznijmy budować graf.

Po pierwsze, trzeba zacząć od definicji symboli wejściowych (np. A,B,C) i wyjśćiowych (np. I,II,III). Do każdej listy symboli trzeba utworzyć objekt `fst.SymbolTable()` i dodać go metodą `add_symbol()`. Metoda ta bierze dwa arguemnty, opis symbolu i identyfikator w postaci liczby. Możesz im przypisać po kolei liczby 1,2,3 (liczba 0 jest zazwyczaj zarezerwowana do symbolu $\epsilon$ reprezentującego pusty symbol).

Potem można stworzyć objekt `fst.FST` i ustawić odpowiednio `set_input_symbols` oraz `set_output_symbols`.

Potem zdefiniujemy 3 stany s0,s1,s2 metodą `add_state`.

W następnej kolejności definiujemy połączenia stanów metodą `add_arc`. Pierwszy argument to stan początkowy połączenia, a drugi to objekt `fst.Arc`. Objekt ten wymaga do skonstrukowania numer symbolu wejściowego, numer symbolu wyjściowego, wagę (ustawmy to na razie na `None`) i stan końcowy połączenia.

Zrób kilka (~5) dowolnych połączeń między powyższymy 3 stanami.

Na samym końcu należy ustawić stan początkowy metodą `set_start` oraz stan(lub stany) końcowe metodą `set_final`. Po uruchomieniu kodu w konteście notebooka, bibliloteka automatycznie wygeneruje wykres reprezentujący utworzony automat.

Zanim przejdziemy do dalszych prac, stwórzmy sobie nowy katalog w którym będą generowane tymczasowe pliki. Stworzymy nowy katalog *grammar* i odnośniki do katalogów `../phonetisaurus`, `../online` oraz `../sejm-audio`:

In [ ]:
%mkdir grammar
%cd grammar
!ln -s ../phonetisaurus
!ln -s ../online
!ln -s ../sejm-audio

### Przygotowanie transkrypcji fonetycznej słów

Pierwszy automat jaki utworzymy będzie służył do konwersji słów na fonemy. Nazwiemy go **L.fst**. Tworzymy go w pierwszej kolejności, bo jest istotne  żebyśmy używali tej samej tablicy słów (identyfikatowów liczbowych) zarówno w leksykonie jak i gramatyce poniżej. Żeby ułatwić ten proces, użyjemy gotowej funkcji. Ściągnij plik https://raw.githubusercontent.com/danijel3/ASRforNLP/main/lexicon.py poleceniem `!wget` i zaimportuj funkcję `words_to_lexicon` z niego:

Do tego zadania przeanalizujemy tylko dwa pliki *powitanie*. Sprawdźmy jakie wyrazy one zawierają. Użyj `!grep` żeby poszukać wyraz *powitanie* w pliku `sejm-audio/text`. Można się też pozbyć identyfikatorów plików, jeśli przekażemy ten wynik do polecenia `cut -f2- -d' '`:

Jeśli teraz dodatkowo ten wynik przekażemy do polecenia `tr ' ' '\n'` a potem do `sort -u` w wyniku otrzymamy unikalną listę słów w transkrypcji:

Skopiujmy te wyrazy do listy w Pythonie i zastosujmy je w funkcji `words_to_lexicon`. Funkcja ta bierze argument w postaci listy słów i zwraca trójkę:
* psyms - lista symboli reprezentująca fonemy (wejście do automatu)
* wsyms - lista symboli reprezentująca wyrazy (wyjście z automatu)
* L - automat reprezentujący leksykon

Po ustawieniu symboli odpowiednio funkcjami `set_inpput_symbols` i `set_output_symbols` można wyświetlić graf:

### Przygotowanie gramatyki

Teraz jak mamy leksykon i listy symboli, możemy przystąpić do definiowania gramatyki. Gramatyka będzie takim automatem której zarówno symbole wejściowe jak i wyjściowe są wyrazami. Formalnie taki automat jest znany jako FSA, czyli Finite State Acceptor.

Gramatyka ma wyglądać docelowo tak:

<img src="https://github.com/danijel3/ASRforNLP/raw/main/grammar.png" width="50%">

Umożliwia ona powiedzenie na początku wyrazu *dziękuję*, ale też przeskoczenie go przejściem "pustym". Potem można powiedzieć jedną z 4 fraz i albo skończyć albo powiedzieć kolejną frazę jeszcze raz. Taka gramatyka powinna sobie poradzić z obydwoma powitaniami wyżej.

Spróbuj odtworzyć gramatykę poleceniami jak wyżej. Pamiętaj żeby ustawić symbole wejściowe i wyjściowe na `wsyms` oraz stany początkowy i końcowy.

Może się przydać następująca funkcja:

```
def add_arc(sf,st,word):
  wid=wsyms.find(word)
  G.add_arc(sf,fst.Arc(wid,wid,None,st))
```

Taki graf jest czytelny i łatwy do interpretacji, ale można zrobić kilka zabiegów po to żeby zajmował mniej zasobów podczas rozpoznawania mowy (pamięci i obliczeń):
* funkcja składowa `rmepsilon` - usuwa puste przejścia z grafu
* funkcja `determinize` - determinizuje graf (usuwa rozgałęzienia wychodzące przyjmujące identyczne tokeny)
* funkcja składowa `minimize` - minimalizuje ilość stanów w grafie

Możemy użyć funkcji `randgen` żeby wygenerować losowe zadania z gramatyki i sprawdzić czy mają sens. Parametr `npath` ustawia ilość zdań do wygenerowania (np. 10), a `max_length` ogranicza długość zdań (też warto ustawić np 10):

Możemy też wziąć przykładowe zdanie i przetestować czy jest ono zgodne z gramatyką. Weźmy jedno zdanie z nagrania i wygenerujmy FST jako pojedynczy łańcuch słów składających się z wyrazów tego zdania, np:
![](https://github.com/danijel3/ASRforNLP/raw/main/sent.png)

In [ ]:
test='dziękuję panie marszałku panie marszałku wysoka izbo panie ministrze'
sent=fst.Fst()
sent.set_input_symbols(wsyms)
sent.set_output_symbols(wsyms)
os=sent.add_state()
sent.set_start(os)
for w in test.split():
    ns=sent.add_state()
    rw=wsyms.find(w)
    sent.add_arc(os,fst.Arc(rw,rw,None,ns))
    os=ns
sent.set_final(ns)
sent

Po dokonaniu kompozycji (funkcją `fst.compose(A,B)`) powyższego zdania z gramatyką dostaniemy jeden z dwóch wyników:
1. jeśli zdanie jest zgodne z gramatyką, dostaniemy to samo zdanie co podaliśmy na wejściu
2. jeśli zdanie nie jest zgodne z gramatyką, dostaniemy pusty graf

### Budowa grafu WFST

Jak zrobiliśmy gramatykę, możemy przystąpić do budowy grafu WFST łączącego wynik modelu akustycznego z procesem rozpoznawania mowy. Graf ten nosi miano HCLG.fst i jest tworzony poprzez kompozycję 4 składowych po kolei.

Ze względów wydajnościowych, zaczynamy budowę grafu od tyłu, więc dokonujemy kompozycji **L** i **G**, po czym od razu dokonujemy determinizacji i minimalizacji wyniku. Wynikowy graf zapiszemy do pliku na dysku funkcją `.write()` pod nazwą `LG.fst`:

**Uwaga:** Następne polecenia uruchamiamy tutaj po kolei żeby dokładnie opisać ich działanie. Podane są tutaj jednak w całości ponieważ łatwo można popełnić błąd w nazwie pliku albo argumentu, a to się mija z celem warsztatów. W praktyce, te polecenia byśmy uruchamiali automatycznie po kolei w pojedynczym skrypcie, gdyż ich zawartość się rzadko zmienia w zależności od danych jakie przetwarzamy.

Kolejnym krokiem jest dodanie kontekstu trifonowego do fonemów, ale zamiast generować osobno automat **C**, Kaldi posiada narzędzie `fstcomposecontext` które dodaje kontekst do dowolnego automatu. Podstawowymi parametrami tego programu są `--contenxt-width` i `--context-position`. Dla wspomnianych wyżej trifonów, pierwszy parametr jest 3, a drugi 1. Nie jest to jednak jedyna dozowolna opcja. Model pentafonowy może mieć parametry 5 i 3. Musimy użyć programu `tree-info` na pliku `online/tree` żeby sprawdzić jakie ustawienie użyto podczas trenowania konkretnego modelu akustycznego:

In [ ]:
!tree-info online/tree

Oprócz kontekstu, dodatkowo należy wczytać `disambig.int` w parametrze `--read-disambig-syms` i zapisać `disambig_ilabels.int` w parametrze `--write-disambig-syms`. Symbole dyzambiguacyjne to specjalne dodatkowe tokeny dodane do modelu po to, żeby procesy optymalizacyjne (np. determinizacja) przebiegały bezproblemowo. Jako argumenty program bierze symbole zapisane w pliku `ilabels` i automat `LG.fst`, a w ostatnim argumencie generuje plik `CLG.fst`:

In [ ]:
!fstcomposecontext --context-size=2 --central-position=1 --read-disambig-syms=disambig.int --write-disambig-syms=disambig_ilabels.int ilabels LG.fst CLG.fst

Ostatnim komponentem jest automat H. On generuje listę ukrytych stanów na podstawie informacji zawartej w modelu akustycznym. On też tworzy własne symbole dyzambiguacyjne przez parametr `--disambig-syms-out` do pliku `disambig_tid.int`. Jako argumenty znowu bierze listę symboli `ilabels` plik z listą ukrytych stanów `online/tree` oraz model akustyczny `online/final.mdl`. Ostatnim argumentem jest wyjściowy plik `H.fst`:

In [ ]:
!make-h-transducer --disambig-syms-out=disambig_tid.int ilabels online/tree online/final.mdl H.fst

Ostatni krok polega na połączeniu automatów H i CLG oraz dokonanie ostatniej determinizacji i minimalizacji. Usuwane są też symbole dyzambigiacyjne automatu H, a oprócz tego dodawane są pętle umożliwiające pozostawanie w jednym stanie ukrytym tyle kroków czasowych ile potrzeba:

In [ ]:
!fsttablecompose H.fst CLG.fst - | fstdeterminizestar --use-log=true - - | fstrmsymbols disambig_tid.int - - | fstminimizeencoded - - | add-self-loops --self-loop-scale=0.1 --reorder=true online/final.mdl - HCLG.fst

Można się zastanowić czemu to jest takie skomplikowane, ale większość nietypowych operacji tutaj wynikają z doświadczania twórców systemu Kaldi i mają na celu zwiększenie wydajności ostatecznego modelu. W skrócie proces jest tak jak opisano w dokumentacji [OpenFST](https://www.openfst.org/twiki/bin/view/FST/PythonExtension):
```
reader = fst.FarReader.open("hclg.far")
LG = fst.determinize(fst.compose(reader["L"], reader["G"]))
CLG = fst.determinize(fst.compose(reader["C"], LG))
HCLG = fst.determinize(fst.compose(reader["H"], CLG))
HCLG.minimize()
HCLG.write("hclg.fst")
```
### Rozpoznawanie mowy

Teraz wreszcze możemy rozpocząć proces rozpoznawania mowy. Do ropznawani mowy potrzebujemy następujące elementy:
* nagrania audio
* model akustyczny
* HCLG.fst

Zacznijmy od przygotowania listy plików audio jakie chcemy rozpoznawać. Stwórzmy plik `wav.scp` z listą plików w następującym formacie:
```
<identyfikator> <ścieżka do pliku>
```
Oprócz tego zróbmy plik `spk2utt` określający mówców:
```
<id mówcy> <id pliku>
```
Ponieważ w naszych plikach są raczej pojedynczy mówcy, zrób żeby nazwa mówcy była taka sama jak nazwa pliku.

Uruchom program `!online2-wav-nnet3-latgen-faster`. Wszystkie standardowe ustawienia procesu rozpoznawania są dodane do modelu akustycznego w prametrze `--config=online/conf/online.conf `. Jedyny niestandardowy parametr to `--word-symbol-table=words.txt`. On nie jest niezbędny do działania programu, ale dzieki niemu program wypisuje wynik rozpoznawania w postaci tekstu w trakcie pracy. Argumentami programu są po kolei:
* `online/final.mdl` - model akustyczny
* `HCLG.fst` - graf WFST który utworzyliśmy wyżej
* `ark:spk2utt` - mapowanie mówców do nagrań
* `scp:wav.scp` - lista plików z nagraniami
* `ark:lat` - wynik rozpoznawania w postaci kraty (o tym za chwilę)

Można zauważyć, że programy w systemie Kaldi czasami przyjmują argumenty w postaci opisu zawierającego prefix `ark:` albo `scp:`. Format SCP służy do opisywania list plików i został wyjaśniony wyżej. Format ARK jest binarnym formatem do przechowywania różnych danych liczbowych - takie archiwum binarne. Później wyjaśnimy jak interpretować dane tam zawarte.

### Sprawdzanie wyniku

Program powyżej służy do generowania wyniku w postaci tzw. kraty - połączonego grfu reprezentującego altermatywne sekwencje słów znalezione przez ASR. Każde połączenie w tym grafie jest ważone, a ścieżka z najlpeszą oceną jest zazwyczaj zwracana jako wynik rozpzonawania. Można użyć programu `lattice-best-path` żeby wypisać najbardziej wiarygodne sekwencje słów dla każdego nagrania. Problem w tym, że wyrazy w naszym modelu HCLG są zakodowane jako integery, więc należy je przekodować w tekst. Do tego się przyda proste narzędzie `/opt/kaldi/egs/wsj/s5/utils/int2sym.pl` z parametrem `-f 2-` i plikiem `words.txt`. Zapiszmy wynik do pliku `trans.txt`:

Na samym końcu, można policzyć jakość rozpoznawania na podstawie miary Word Error Rate (WER) programem `compute-wer`. Program ten bierze jako pierwsy argument tekst z refencją w pliku `ark:sejm-audio/text` a jako drugi wynik rozpoznawania, który wyliczyliśmy wyżej, czyli `ark:trans.txt` Ponieważ rozpoznaliśmy tylko 2 z 12 plików w referencji, musimy również dodać parametr `--mode=present` żeby policzyć błąd tylko dla plików obecnych w wyniku rozpoznawania. Jeśli wszystko jest w porządku, wynik powinien wynosić 0%.

Podejście gramatyczne jest bardzo poteżne i umożliwia pełną kontrolę nad procesem rozpoznawania. W dodatku dokładnie demonstruje proces i rolę poszczególnych elementów grafu do rozpoznawania mowy. Niestety, stworzenie czegokolwiek bardziej złożonego od rozpoznawania kilku zdań robi się niepraktyczne. Są pewne próby ułatwienia tego procesu jak projekt [Thrax](https://www.openfst.org/twiki/bin/view/GRM/Thrax), ale nadal jest to skomplikowane i wymaga sporo pracy.

## Podejście oparte o statystyczny model języka

Innym podejściem jest wygenerowanie grafu na podstawie statystyk uzyskanych na dużym zbiorze tekstów. Standardowo do tego używamy pojęcie modelu n-gramowego gdzie modelujemy n-elementową zbitkę słów następujących po sobie. W rozpoznawaniu mowy najczęściej stosujemy model trigramowy, który modeluje trójki słów. Ponieważ nasz graf G.fst obsługuje wagi na przejściach między stanami, taki model n-gramowy można stosunkowo łatwo zaprezentować w postaci grafu WFST.

Utowrzymy najpierw katalog do przechowywania wszystkich plików tymczasowych. Zrób katalog `/content/lm` i przejdź do niego, a potem dodaj odnośniki do katalogów `../phonetisaurus`, `../online`, `../sejm-audio` oraz pliku `../sejm-text`:

In [ ]:
%cd /content
%mkdir lm
%cd lm
!ln -s ../phonetisaurus
!ln -s ../online
!ln -s ../sejm-audio
!ln -s ../sejm-text

### Podstawy modelu n-gramowego

Model języka jest trenowany na zbiorze przykładowych zdań. Stwórzmy plik zawierający kilka przykładowych zdań. Użyj polecenia `%%writefile test.txt` żeby stworzyć plik z jednym zdaniem w linii: "*ala ma kota*", "*ala ma psa*" i "*jan ma kota*":

Żeby wygenerować model języka, użyjemy programu z pakietu [SRILM](http://www.speech.sri.com/projects/srilm/). Nie jest to jedyne narzędzie do trenowania modeli języka, ale ma bardzo bogatą historię i mnóstwo zaimplementowanych mechanizmów. Inne narzędzia o tych samych zastosowania są przykładowo: MITLM, IRSTLM, KenLM, PocoLM.

Program `ngram-count` bierze następujące parametry:
* `-text test.txt` określa plik z źródłem danych
* `-order 3` mówi o rzędzie modelu - w tym przypadku jest to model trigramowy
* `-wbdiscount` to metoda umożliwiająca modelowi radzenie sobie z danymi nie występującymi w danych treningowych - jest kilka alternatywych metod, ale Witten-Bell jest optymalny dla bardzo małych zbiorów danych
* `-lm out.arpa` określa plik wynikowy

Plik ARPA jest formatem tekstowym i można łatwo odczytać jego zawartość. Użyj polecenia `%cat out.arpa` żeby go wyświetlić:

Składa się on z nagłówka zaczynającego od tokenu `/data/` i zawierającego liczność poszczególnych n-gramów. Potem mamy kolejne sekcje, każda zawierająca listę poszczególnych n-gramów.

Każdy n-gram jest opisany dwoma lub trzema polami oddzielonymi znakami `\t`:
* prawdopodobieństwo danego n-gramu w skali logarytmicznej
* opis samego n-gramu (tokeny/słowa oddzielone spacją)
* opcjonalnie tzw. "*back-off weight*" też w skali log

Back-off jest metodą do określenia prawdopodobieństwa n-gramów wyższego stopnia użwyając n-gramów niższego. Z tego powodu, najwyższe n-gramy (w naszym przypadku 3-gramy) nie mają policzonych wag back-off. Algorytm liczenia prawdopodonieństwa n-gramu jest następujący:

* jeśli na liście jest dokładnie ten n-gram którego szukamy, bierzemy jego prawdopodobieństwo
* jeśli go nie ma liście, bierzemy prawdopodobieństwo według wzoru:

\begin{equation}
P( word_N | word_{N-1}, word_{N-2}, ...., word_1 ) = \\
P( word_N | word_{N-1}, word_{N-2}, ...., word_2 ) \cdot \text{backoff-weight}(  word_{N-1} | word_{N-2}, ...., word_1 )
\end{equation}

* jeśli brakuje prawdopodobieństwa n-gramu mniejszego stopnia, wtedy rekurencyjnie stosujemy ten sam wzór aż do unigramów (które wszystkie powinny być zdefiniowane)
* jeśli brakuje wagi back-off, zakładmy wartość 1 (czyli 0 w skali logarytmicznej)

Na przykład, prawdopodobieństwo n-gramu "*ala ma*" jest następujące:

\begin{equation}
P(ma|ala) = 10^{-0.1760913} = 0.6666666038148176
\end{equation}

A prawdopodobieństwo n-gramu "*jan ma psa*":


\begin{equation}
P(psa|jan,ma) = P(psa|ma)*bwt(ma|jan)=10^{(-0.69897+0)}=0.20000000199681048
\end{equation}

Użyjmy prostej biblioteki `arpa` żeby potwierdzić powyższe obliczenia. Dokumentacja do biblioteki jest [tutaj](https://pypi.org/project/arpa/). Zainstaluj bibliotekę `arpa` poleceniem `!pip install`, a potem ją zaimportuj. Funkcja `arpa.loadf()` służy do wczytania modelu. Funkcja ta zwraca listę (standard ARPA widocznie wspiera więcej modeli w jednym pliku), więc należy odczytać pierwszy element z wynikowej listy. Potem w tym modelu można użyć funkcji `.p()` żeby odczytać prawdopodobieństwo n-gramu  lub funkcję `.s()` żeby odczytać sumaryczne prawdopodobieństwo zdania. Są też odpowiednie funkcje w skali logarytmicznej (wskazane dla dłuższych tekstów):

Jedną z podstatowych miar jakości modelu języka to tzw. *perplexity*. Liczymy go stotując wytrenowany model języka na niezależnym zbiorze testowym. Zróbmy przykładowy zbiór zawierający jedno zdanie (np. *ala ma osę*) i zapiszmy w pliku `eval.txt`:

Do wyliczenia perplexity użyjemy programu `ngram` i użyjemy w nim opcję `-lm out.arpa` do wczytania pliku z modelem oraz `-ppl eval.txt` żeby policzyć perplexity na wybranym pliku:

Wyniki zawierają ilość zdań, słów i wyrazów spoza słownika (OOV - out-of-vocabulary). Zawiera też wyliczony logprob całego korpusu oraz perplexity wyczlione uwzględniając (`ppl1`) i nieuwzlgędniając (`ppl`) sztucznie dodane tokeny `<s>` oraz `</s>`. Im mniejsza wartość PPL, tym model lepiej opisuje testowy zbiór tekstów.

Program `ngram` ma mnóstwo zastosowań, głównie związanych z edycją i manipulacją wytrenowanego modelu języka. Ma też opcję `-gen <N>` do wygenerowania losowych zdań z konkretnego modelu języka. Użyj go żeby wygenerować 10 losowych zdań:

Użyjmy teraz narzędzia `arpa2fst` do wygenerowania transducera odpowiadającego powyższemu modelu języka:

Użyjmy naszej bibliotegko OpenFST do wczytania (metodą `fst.FST.read()`) i narysowania grafu modelu:

### Wytrenowanie modelu na większej ilości tekstu

Wytrenujmy więc model języka, który sobie poradazi ze wszystkimi nagraniami z naszego małego zbioru. Użyjemy do tego pliku `sejm-text`, który ściągnęliśmy na samym początku. W programie `ngram-count` użyjemy następujące opcje:
* `-order 3` - chcemy 3-gramowy model języka
* `-unk` - model ma zawierać token UNK do modelowania słów spoza słownika (OOV)
* `-kndiscount` - użyjemy metody wygładzania Knesser-Ney
* `-text sejm-text` - korpus tekstowy
* `-write-vocab word.list` - zapisz listę słów do pliku
* `-lm sejm.arpa` - zapisz model do pliku `sejm.arpa`

Po stowrzeniu pliku użyj polecenia `!gzip sejm.arpa` żeby skompresować model, żeby zajmował trochę mniej miejsca na dysku. Programy z SRILM bez problemu wczytują pliki skompresowane metodą *gzip*.

Wygenerujemy przykładowe zdania z tego modelu języka:

Policzymy perplexity transkrypcji naszych nagrań na wytrenowanym modelu języka. Warto pamiętać o zastosowaniu polecenia `!cut -f2- -d' '` na transkrypcji, żeby ocenić tylko tekst, bez identyfikatorów plików:

### Utworzenie trankrypcji fonetycznej

Zaczniemy więc od stworzwnia leksykonu do nowego modelu języka. Wczytajmy listę słów z pliku `word.list`, ale przskoczymy niektóre specjalne słowa których nie chcemy mieć w leksykonie (`-pau-`, `<unk>`, `<s>`, `</s>`):

In [ ]:
words=[]
with open('word.list') as f:
  for l in f:
    w=l.strip()
    if w!='-pau-' and w[0]!='<':
      words.append(w)
psyms,wsyms,L=words_to_lexicon(words)

Zapiszmy tym razem leksykon do pliku `L.fst` na dysku:

Żeby ułatwić proces deteriminizacji później, dodamy pętle do specjalnych tokenów dysambiguacyjnychm zarówno po stronie fonemów, jak i słów. Najpierw musimy znaleźć liczby reprezentujące te tokeny (występujące pod nazwą `#0` w listach symboli - można do tego użyć programu `grep` na plikach `phones.txt` i `words.txt`), a potem użyjemy programu `fstaddselfloops`. Dodatkowo możemy posortować przejścia programem `fstarcsort` ustawiając parametr `--sort_type=olabel` i wynik zapiszemy w pliku `L_disambig.fst`:

In [ ]:
!grep '#0' phones.txt | cut -f2 -d' ' > wdisambig_phones.int
!grep '#0' words.txt | cut -f2 -d' ' > wdisambig_words.int
!fstaddselfloops wdisambig_phones.int wdisambig_words.int < L.fst | fstarcsort --sort_type=olabel > L_disambig.fst

### Budowa grafu WFST

Użyjmy teraz program `arpa2fst` żeby wygenerować automat `G.fst`. Dodamy do programu parametr `--disambig-symbol="#0"` oraz wczytamy listę symboli z pliku `--read-symbol-table=words.txt`:

In [ ]:
!gunzip -c sejm.arpa.gz | arpa2fst --disambig-symbol="#0" --read-symbol-table=words.txt - G.fst

Teraz możemy dokonać kompozycji `L_disambig.fst` i `G.fst`, a potem zastosować determinizacje programem `fstdeterminizestar` ustawiając `--use-log=true` i minimalizacje programem `fstminimizeencoded`. Dodatkowo możemy użyć operacji przesuwania symboli programem `fstpushspecial`, żeby dodatkowo zoptymalizować działanie automatu:

In [ ]:
!fsttablecompose L_disambig.fst G.fst | fstdeterminizestar --use-log=true | fstminimizeencoded | fstpushspecial > LG.fst

Tak jak poprzednio, dodajemy kontekst żeby utworzyć automat `CLG.fst`:

In [ ]:
!fstcomposecontext --context-size=2 --central-position=1 --read-disambig-syms=disambig.int --write-disambig-syms=disambig_ilabels.int ilabels LG.fst | fstarcsort --sort_type=ilabel > CLG.fst

Generujemy automat `H.fst`:

In [ ]:
!make-h-transducer --disambig-syms-out=disambig_tid.int --transition-scale=1.0 ilabels online/tree online/final.mdl H.fst

I dokonujemy ostatecznej kompozycji żeby utworzyć finalny model `HCLG.fst`:

In [ ]:
!fsttablecompose H.fst CLG.fst - | fstdeterminizestar --use-log=true - - | fstrmsymbols disambig_tid.int - - | fstminimizeencoded - - | add-self-loops --self-loop-scale=0.1 --reorder=true online/final.mdl - HCLG.fst

Warto zwrócic uwagę na rozmiar automatu poleceniem `%ls -lh`. Jest on znacznie większy niż poprzedni automat oparty na ręcznie napisanej gramatyce. Jest też znacznie większy niż model ARPA. Warto wziąć pod uwagę, że automat HCLG.fst będzie w bardzo przybliżonym rozmiarze wczytany do pamięci RAM podczas rozpoznawania mowy. W prkatycznych zastosowaniach (ze słownikami kilkudzisięciu-klikuset tysięcy słów) ten model zajmuje największy udział w pamięci procesu rozpoznawania mowy (kilkaset MB do kilka GB):

### Rozpoznawanie mowy

Podobnie jak wyżej, zróbmy listy plików `wav.scp` i `spk2utt`, ale tym razem uwzględnijmy cały katalog `sejm-audio`:

Podobnie jak wyżej, użyjemy `!online2-wav-nnet3-latgen-faster` do rozpoznawania:

### Sprawdzenie wyników

Tak jak wcześniej, wygnerujmy listy wyników:

I sprawdźmy co zostało rozpoznane poleceniem `%cat trans.txt`:

Policzmy WER:

Możemy też użyc program `align-text` i skryptu `/opt/kaldi/egs/wsj/s5/utils/scoring/wer_per_utt_details.pl` żeby dostać dokładny opis błędów:

Wróćmy jednak do pojęcia kraty - krata to połączony graf zawierający wszystkie alternatywy procesu rozpoznawania mowy. Możemy ją obejrzeć jeśli skopiujemy jej zawartość do postaci tekstowej programem `lattice-copy` (zamieniając `ark:lat` na `ark,t:-`) i w wyniku zamienimy wartości w 3 kolumnie na wyrazy skryptem `int2sym.pl`:

Format tego pliku zawiera następujące kolumny:
* stan początkowy
* stan docelowy
* wyraz
* waga - definicja wagi jest:
  * waga modelu języka
  * waga modelu akustycznego
  * sekwencja stanów ukrytych (H) reprezentujących dany wyraz

Najlepsza sekwencja jest taka która ma najwyższą ocenę względem sumy (w skali log, czyli iloczynu normalnie) dwóch prawdopodobieństw (języka i akustyki).

Używając programu `lattice-to-fst` możemy uprościć kratę a potem poleceniem `sed -n '/AndrzejKania.*/,/^$/p' | tail -n +2)` wyciągnąć poojedynczą kratę oraz zamienić ją na FST programem `fstcompile` i wyświetlić podobnie do G.fst wyżej.

Ponieważ krata ta jest trochę duża, warto ją jednak zapisać do pliku poleceniem `lattice.draw('lattice.dot')` a potem użyć programu `dot -Tpng -Gdpi=1800 lattice.dot > lattice.png` żeby ją zapisać do pliku. Taki plik możemy ściągnąć z przeglądarki plików po lewej i obejrzeć lokalnie na komputerze.

Najlepsza ścieżka względem oceny modeli akustycznego i języka w każdej kracie generuje odpowiedź o określonym wyżej WER, ale to wcale nie oznacza, że jest ona najbardziej poprawna wzgledem prawdziwego nagrania. Gdybyśmy mieli wyrocznie, która nam podowiada jakie wyrazy bardziej pasują do prawdziwego nagrania, moglibyśmy w kracie znaleźć lepsze rozwiązanie. Program `lattice-oracle` służy do tego żeby ocenić tą potencjalną ocenę. Uruchom polecenie:
```
!lattice-oracle ark:lat ark,t:"/opt/kaldi/egs/wsj/s5/utils/sym2int.pl -f 2- words.txt < sejm-audio/text|" ark:/dev/null
```
Warto zauważyć, że wynik ten można poprawić zmieniając parametry `--beam` i `--lattice-beam` w konfiguracji dekodera (plik `online/conf/online.conf`).


Zamiast kraty, możemy też pracować na listach N-best. To jest lista N najlepszych ścieżek w całej kracie. Używając programu `lattice-to-nbest` możemy wygenerować takie listy dla poszczególnych plików, a programem `nbest-to-linear` zamienić je na ciągi słów. Znowu, skrypt `int2sym.pl` się przydaje żeby zamienić liczby na słowa:

## Inne pomysły

Co dalej możemy osiągnąć z tą wiedzą? Tu jest krótka lista pomysłów do czego się używa krat w analizie transkrypcji mowy:

* poprawa jakości rozpoznawania mowy - najczęściej przez tzw. rescoring. Używamy innego, bardziej rozbudowanego modelu żeby zmienić wagi w kracie i znaleźć lepszą hipotezę od wstępnej.
* wyszukiwanie - znajdywanie wyrazów, czy innych informacji w nagraniach mowy jest nieco skuteczniejsze jeśli uwzglednimy alternatywy w kracie albo liście nbest
* analiza dyskursu - jeśli chcemy poprawnie analizować dialog z nagrania mowy, musimy wykorzystać skomplikowane modele NLP. Analiza ta nie jest łatwa i stosowanie kraty może zwiększyć szanse na uzyskanie poprawnego wyniku. Większość istniejących rozwiązań nie jest przystosowana do takiego trybu pracy.
* polecenia - podobnie jak analiza dialogu człowieka z człowiekiem, rozmowa człowieka z komputerem bywa bardzo skomplikowana. Tutaj stosowanie mieszanki modeli statystycznych i ręcznie napisanych modeli może być bardzo pomocna.
* tłumaczenie/tranformacja - przekształcanie tekstu z jednej postaci na inną jest ciekawym zagadnieniem - oczywiście między językami, ale rónwież na przykład z tekstu na obraz.

I to jest dopiero początek. Mowa się coraz częściej staje dodatkowym źródłem informacji w skomplikowanych modelach. Łatwo można sobie wyobrazić wielo-modalne systemy wykorzystujące jednocześnie dźwięk, obraz i wiele innych źródeł informacji żeby realizować złożone zadania. Niestey, większość obecnych narzędzi NLP jest wciąż nieprzystosowana do pracy z danymi konwersacyjnymi, więc zachęchamy do badań w tym kierunku.